# Bellabeat Case Study
Author: Brent Artuch <br>
Date: 2023-06-13

## Introduction

The client, Bellabeat, is a retailer of fitness goods focused on women's health who is looking
for assistance with analysis of the trends of the use of smart devices for women's fitness and
well-being. After analyzing the trends in the given dataset, the client would like recommendations
on how to interpret and apply these trends to future business decisions.

This scenario is one of the Google Data Analytics Capstone projects from the professional
certification track. The dataset is provided by Kaggle user Mobius and is licensed under CC0:
Public Domain.

This project is developed in the Jupyter Notebook environment but the key focus of the execution
of this analysis will be to demonstrate the capabilities of SQL. Pandas data frames will be used
for tabular display of the resulting queries and Tableau will be utilized for the data
visualizations.

## Step 1: Imports
Import Pandas and SQLite.

In [1]:
import sqlite3
import pandas as pd
from IPython.display import Image

### Load the datasets
The datasets have been loaded into SQLite database using the workbench, but we will output the
first 5 entries of each to get an idea of their scope.

In [2]:
sql_connection = sqlite3.connect('bellabeat.sqlite')
pd.read_sql('SELECT * FROM dailyActivity_merged LIMIT 5', sql_connection)

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,4/12/2016,13162,8.50,8.50,0,1.88,0.55,6.06,0.0,25,13,328,728,1985
1,1503960366,4/13/2016,10735,6.97,6.97,0,1.57,0.69,4.71,0.0,21,19,217,776,1797
2,1503960366,4/14/2016,10460,6.74,6.74,0,2.44,0.40,3.91,0.0,30,11,181,1218,1776
3,1503960366,4/15/2016,9762,6.28,6.28,0,2.14,1.26,2.83,0.0,29,34,209,726,1745
4,1503960366,4/16/2016,12669,8.16,8.16,0,2.71,0.41,5.04,0.0,36,10,221,773,1863


In [3]:
pd.read_sql('SELECT * FROM hourlyCalories_merged LIMIT 5', sql_connection)

,Id,ActivityHour,Calories
0,1503960366,4/12/2016 12:00:00 AM,81
1,1503960366,4/12/2016 1:00:00 AM,61
2,1503960366,4/12/2016 2:00:00 AM,59
3,1503960366,4/12/2016 3:00:00 AM,47
4,1503960366,4/12/2016 4:00:00 AM,48


In [4]:
pd.read_sql('SELECT * FROM hourlyIntensities_merged LIMIT 5', sql_connection)

,Id,ActivityHour,TotalIntensity,AverageIntensity
0,1503960366,4/12/2016 12:00:00 AM,20,0.333333
1,1503960366,4/12/2016 1:00:00 AM,8,0.133333
2,1503960366,4/12/2016 2:00:00 AM,7,0.116667
3,1503960366,4/12/2016 3:00:00 AM,0,0.000000
4,1503960366,4/12/2016 4:00:00 AM,0,0.000000


In [5]:
pd.read_sql('SELECT * FROM sleepDay_merged LIMIT 5', sql_connection)

,Id,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1.503960e+09,4/12/2016 12:00:00 AM,1,327,346
1,1.503960e+09,4/13/2016 12:00:00 AM,2,384,407
2,1.503960e+09,4/15/2016 12:00:00 AM,1,412,442
3,1.503960e+09,4/16/2016 12:00:00 AM,2,340,367
4,1.503960e+09,4/17/2016 12:00:00 AM,1,700,712


In [6]:
pd.read_sql('SELECT * FROM weightLogInfo_merged LIMIT 5', sql_connection)

,Id,Date,WeightKg,WeightPounds,Fat,BMI,IsManualReport,LogId
0,1.503960e+09,5/2/2016 11:59:59 PM,52.599998,115.963147,22.0,22.650000,True,1.462234e+12
1,1.503960e+09,5/3/2016 11:59:59 PM,52.599998,115.963147,NaN,22.650000,True,1.462320e+12
2,1.927972e+09,4/13/2016 1:08:52 AM,133.500000,294.317120,NaN,47.540001,False,1.460510e+12
3,2.873213e+09,4/21/2016 11:59:59 PM,56.700001,125.002104,NaN,21.450001,True,1.461283e+12
4,2.873213e+09,5/12/2016 11:59:59 PM,57.299999,126.324875,NaN,21.690001,True,1.463098e+12


## Step 2: Data Cleaning
We will create separate `date` and `time` columns for easier comparisons.

In [7]:
dt_hourly_cals = pd.read_sql(
    """
    SELECT Id, trim(substr(ActivityHour, 1, 9)) AS Date, trim(substr(ActivityHour, 10)) AS Time,
    Calories FROM hourlyCalories_merged
    """, sql_connection)
dt_hourly_cals.head()

,Id,Date,Time,Calories
0,1503960366,4/12/2016,12:00:00 AM,81
1,1503960366,4/12/2016,1:00:00 AM,61
2,1503960366,4/12/2016,2:00:00 AM,59
3,1503960366,4/12/2016,3:00:00 AM,47
4,1503960366,4/12/2016,4:00:00 AM,48


In [8]:
dt_hourly_int = pd.read_sql(
    """
    SELECT Id, trim(substr(ActivityHour, 1, 9)) AS Date, trim(substr(ActivityHour, 10)) AS Time,
    TotalIntensity, AverageIntensity FROM hourlyIntensities_merged
    """, sql_connection)
dt_hourly_int.head()

,Id,Date,Time,TotalIntensity,AverageIntensity
0,1503960366,4/12/2016,12:00:00 AM,20,0.333333
1,1503960366,4/12/2016,1:00:00 AM,8,0.133333
2,1503960366,4/12/2016,2:00:00 AM,7,0.116667
3,1503960366,4/12/2016,3:00:00 AM,0,0.000000
4,1503960366,4/12/2016,4:00:00 AM,0,0.000000


In [9]:
dt_sleep = pd.read_sql(
    """
    SELECT Id, trim(substr(SleepDay, 1, 9)) AS Date, trim(substr(SleepDay, 10)) AS Time,
    TotalSleepRecords, TotalMinutesAsleep, TotalTimeInBed FROM SLeepDay_merged
    """, sql_connection)
dt_sleep.head()

,Id,Date,Time,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1.503960e+09,4/12/2016,12:00:00 AM,1,327,346
1,1.503960e+09,4/13/2016,12:00:00 AM,2,384,407
2,1.503960e+09,4/15/2016,12:00:00 AM,1,412,442
3,1.503960e+09,4/16/2016,12:00:00 AM,2,340,367
4,1.503960e+09,4/17/2016,12:00:00 AM,1,700,712


### Check for Missing Data
Next we will check our datasets for any null values and drop them.

In [10]:
# Utilize the UNION keyword to create a dataframe with column and row headers.
pd.read_sql(
    """
    SELECT 'dailyActivity' AS TableName, count(*) AS NumNullRows FROM dailyActivity_merged
    WHERE TotalSteps is null OR TotalDistance is null OR SedentaryMinutes is null OR
    Calories is null
    UNION
    SELECT 'hourlyCalories', count(*) AS NumNullRows FROM hourlyCalories_merged
    WHERE Calories is null
    UNION
    SELECT 'hourlyIntensities', count(*) AS NumNullRows FROM hourlyIntensities_merged
    WHERE TotalIntensity is null OR AverageIntensity is null
    UNION
    SELECT 'sleepDay', count(*) AS NumNullRows FROM sleepDay_merged
    WHERE TotalMinutesAsleep is null OR TotalTimeInBed is null
    UNION
    SELECT 'weightLogInfo', count(*) AS NumNullRows FROM weightLogInfo_merged
    WHERE WeightPounds is null OR Fat is null OR BMI is null
    """,
    sql_connection).set_index('TableName')

,NumNullRows
TableName,
dailyActivity,0
hourlyCalories,0
hourlyIntensities,0
sleepDay,0
weightLogInfo,65


Since there are already four null values for the `Fat` column displayed by the head function for
the `weightLogInfo` table, we'll exclude that column to see if resolves the null values.

In [11]:
new_weight_log = pd.read_sql(
    """
    SELECT Id, Date, WeightKg, WeightPounds, BMI
    FROM weightLogInfo_merged
    """, sql_connection)
# Call the pandas functions to count NaN values for the new DataFrame.
new_weight_log.isna().sum()

Id              0
Date            0
WeightKg        0
WeightPounds    0
BMI             0
dtype: int64

### Export To SQL
Next we will export the newly created tables with the new `date` and `time` columns to be stored
in SQL as new tables. The cleaned `weight_log` table is also added.

In [12]:
dt_hourly_cals.to_sql('dt_hourly_cals', sql_connection, if_exists='replace', index=False)
dt_hourly_int.to_sql('dt_hourly_int', sql_connection, if_exists='replace', index=False)
dt_sleep.to_sql('dt_sleep', sql_connection, if_exists='replace', index=False)
new_weight_log.to_sql('new_weight_log', sql_connection, if_exists='replace', index=False);

## Step 3: Explore the Data
Now lets check for the number of unique ID numbers in each dataset, so that we know the number of
active participants.

In [13]:
pd.read_sql(
    """
    SELECT 'dailyActivity' AS TableName, count(DISTINCT Id) AS NumUsers FROM dailyActivity_merged UNION
    SELECT 'hourlyCalories', count(DISTINCT Id) FROM dt_hourly_cals UNION
    SELECT 'hourlyIntensities', count(DISTINCT Id) FROM dt_hourly_int UNION
    SELECT 'sleepDay', count(DISTINCT Id) FROM dt_sleep UNION
    SELECT 'weightLog', count(DISTINCT Id) FROM new_weight_log
    """, sql_connection)

,TableName,NumUsers
0,dailyActivity,33
1,hourlyCalories,33
2,hourlyIntensities,33
3,sleepDay,24
4,weightLog,8


Since there are so few participants in the weight log table, we will disregard it from the study
due insufficient sample size. Next will get some summary statistics of the datasets starting with the
`dailyActivity` dataset.

In [14]:
# Utilize the UNION keyword to create a dataframe with column and row headers.
pd.read_sql(
    """
    SELECT 'TotalSteps' AS Name, round(avg(TotalSteps)) AS Mean, round(min(TotalSteps)) AS Min,
    round(max(TotalSteps)) AS Max FROM dailyActivity_merged
    UNION
    SELECT 'TotalDistance', round(avg(TotalDistance),1), round(min(TotalDistance),1),
    round(max(TotalDistance),1) FROM dailyActivity_merged
    UNION
    SELECT 'SedentaryMinutes', round(avg(SedentaryMinutes),1), round(min(SedentaryMinutes),1),
    round(max(SedentaryMinutes),1) FROM dailyActivity_merged
    UNION
    SELECT 'Calories', round(avg(Calories),1), round(min(Calories),1),
    round(max(Calories),1) FROM dailyActivity_merged
    UNION
    SELECT 'VeryActiveMinutes', round(avg(VeryActiveMinutes),1), round(min(VeryActiveMinutes),1),
    round(max(VeryActiveMinutes),1) FROM dailyActivity_merged
    UNION
    SELECT 'FairlyActiveMinutes', round(avg(FairlyActiveMinutes),1), round(min(FairlyActiveMinutes),1),
    round(max(FairlyActiveMinutes),1) FROM dailyActivity_merged
    UNION
    SELECT 'LightlyActiveMinutes', round(avg(LightlyActiveMinutes),1), round(min(LightlyActiveMinutes),1),
    round(max(LightlyActiveMinutes),1) FROM dailyActivity_merged
    """,
    sql_connection).set_index('Name')

,Mean,Min,Max
Name,,,
Calories,2303.6,0.0,4900.0
FairlyActiveMinutes,13.6,0.0,143.0
LightlyActiveMinutes,192.8,0.0,518.0
SedentaryMinutes,991.2,0.0,1440.0
TotalDistance,5.5,0.0,28.0
TotalSteps,7638.0,0.0,36019.0
VeryActiveMinutes,21.2,0.0,210.0


Now `dt_hourly_cals`:

In [15]:
pd.read_sql(
    """
    SELECT 'Calories' AS Name, round(avg(Calories)) AS Mean, round(min(Calories)) AS Min,
    round(max(Calories)) AS Max FROM dt_hourly_cals
    """, sql_connection).set_index('Name')

,Mean,Min,Max
Name,,,
Calories,97.0,42.0,948.0


And finally `dt_sleep`

In [16]:
pd.read_sql(
    """
    SELECT 'TotalMinutesAsleep' AS Name, round(avg(TotalMinutesAsleep), 1) AS Mean, round(min(TotalMinutesAsleep), 1) AS Min,
    round(max(TotalMinutesAsleep), 1) AS Max FROM dt_sleep
    """, sql_connection).set_index('Name')

,Mean,Min,Max
Name,,,
TotalMinutesAsleep,419.5,58.0,796.0


### Merge Tables
Before moving on to visualizations, I am going to merge the tables so that they are easier to
work with in Tableau. First we will merge the `dailyActivity_merged` and `dt_sleep` tables.

In [17]:
new_activity = pd.read_sql(
    """
    SELECT DA.Id, DA.ActivityDate AS Date, DA.TotalSteps AS Steps, DA.TotalDistance AS
    Distance, DA.SedentaryMinutes, DA.Calories, S.TotalMinutesAsleep
    FROM dailyActivity_merged AS DA
    INNER JOIN dt_sleep AS S ON DA.Id = S.Id AND DA.ActivityDate = S.Date
    """, sql_connection)
new_activity.head()

,Id,Date,Steps,Distance,SedentaryMinutes,Calories,TotalMinutesAsleep
0,1503960366,4/12/2016,13162,8.50,728,1985,327
1,1503960366,4/13/2016,10735,6.97,776,1797,384
2,1503960366,4/15/2016,9762,6.28,726,1745,412
3,1503960366,4/16/2016,12669,8.16,773,1863,340
4,1503960366,4/17/2016,9705,6.48,539,1728,700


And then `dt_hourly_cals` and `dt_hourly_int` as `hourly_activity`.

In [18]:
hourly_activity = pd.read_sql(
    """
    SELECT HI.Id, HI.Date, HI.Time, HI.TotalIntensity, HI.AverageIntensity, HC.Calories
    FROM dt_hourly_int AS HI
    INNER JOIN dt_hourly_cals AS HC ON HI.Id = HC.Id AND HI.Date = HC.Date AND
    HI.Time = HC.Time
    """, sql_connection)
# A little help from pandas with hour format since SQLite lacks that functionality.
hourly_activity['Time'] = pd.to_datetime(hourly_activity['Time']).dt.strftime('%H:%M')
hourly_activity.head()

,Id,Date,Time,TotalIntensity,AverageIntensity,Calories
0,1503960366,4/12/2016,00:00,20,0.333333,81
1,1503960366,4/12/2016,01:00,8,0.133333,61
2,1503960366,4/12/2016,02:00,7,0.116667,59
3,1503960366,4/12/2016,03:00,0,0.000000,47
4,1503960366,4/12/2016,04:00,0,0.000000,48


### Export Visualization Tables to CSV

In [19]:
new_activity.to_csv("Datasets/DailyActivity.csv")
hourly_activity.to_csv("Datasets/HourlyActivity.csv")

## Step 4: Visualize the Data

In [20]:
Image(url="Visualizations/Calories_v_Steps.png", width=1200, height=600)

This figure suggests a decently strong positive correlation between the amount of steps taken in a day and the
amount of calories burned. The figure suggests a p-value of roughly 0.0001 so the relationship is statistically significant.

In [21]:
Image(url="Visualizations/SleepDistribution.png", width=1200, height=600)

The histogram above illustrates that the majority of the users are getting between six and eight hours of sleep per night with some exceptions
on both extremes of the distribution. Overall, the figure is normally distributed.

In [22]:
Image(url="Visualizations/IntensityVCalories.png", width=1200, height=600)

This scatter plot shows an incredibly strong correlation between intensity levels and calorie burn which is logically what we would assume,
but this absolutely confirms that null hypothesis.

In [23]:
Image(url="Visualizations/HourlyActivity.png", width=1200, height=600)

And finally this bar plot shows the average intensity trend as the day progresses, and it looks worth noting that the participants are showing
the highest intensity levels from 5 PM to 7 PM.

## Conclusion
So our client the original question our client wanted to be answered was "how are their customers using smart devices to monitor
overall fitness and well-being?"
<br>
<b> Step Counting: </b>
As shown by the analysis, the number of steps one takes during a day is a proportion to the amount of calories that said individual
will burn. So, implementing step counting capabilities into smart devices and focusing on that feature as a selling point will lead
to greater interest among the general customer base.
<br>
<b> Sleep Tracking: </b>
Having a sleep tracker in devices and metrics pertaining to the average sleep times of other users could be another incentive
for the customer to be interested in a smart device. Getting adequate sleep is known to lead to many health benefits such as
mood, energy levels, and focus.
<br>
<b> Intensity Tracking: </b>
By far the strongest evidence uncovered in this analysis was the strong correlation between intensity levels and daily caloric
burn. If the smart devices could calculate an intensity level for the user that is limited to discrete quantities, the user
could then predict caloric burn based on time and intensity. This would allow the user to gain a level of control over their
ability to determine caloric burn with the help of their smart device.
<br>
<b> Hourly Tracking: </b>
In tandem with intensity tracking is the user's ability to track their intensity levels on an hourly basis. With this information,
the user could decide when it is the best time to eat certain meals or avoid certain strenuous activities because it may have
an effect on their exercise intensity.
<br>
Combing all of these elements would result in a smart device that would give the user thorough information and forcasting
of their burning of and ability to burn calories.
